In [2]:
import json
import pandas as pd
import os
import numpy as np

In [ ]:
df = pd.read_csv("aws_dea_questions_final.csv")

# Biến đổi các kí tự "/n/n" thành "<br><br>"
df['question_text'] = df['question_text'].str.replace('/n/n', '<br><br>')
df['question_text'] = df['question_text'].str.replace('\n\n', '<br><br>')

In [20]:
df = pd.read_csv("aws_dea_questions_final.csv")

# Biến đổi các kí tự "/n/n" thành "<br><br>"
df['question_text'] = df['question_text'].str.replace('\n\n', '<br><br>')

df = df [['question_text', 'correct_answer', 'A', 'C', 'D', 'B', 'E']]

type1 = df[df['correct_answer'].str.len() == 1]
type2 = df[df['correct_answer'].str.len() > 1]

type1_split = np.array_split(type1, 23)
type2_split = np.array_split(type2, 23)

file_order = [0] * len(df)
for i in range(23):
    idx_type1 = type1_split[i].index.tolist()
    idx_type2 = type2_split[i].index.tolist()
    for idx in idx_type1 + idx_type2:
        file_order[idx] = i + 1

df['exam'] = file_order

/home/phongnl/exam_DEA/Quiz_app/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


# Convert to json

In [21]:
for exam_num in range(1, 24):
    df_subset = df[df['exam'] == exam_num]
    file_name = f'questions_{exam_num}.json'
    if df_subset.empty:
        continue
    questions_list = []
    for _, row in df_subset.iterrows():
        # Lấy tất cả các cột chứa đáp án (A, B, C, D, E,...)
        options_cols = [col for col in row.index if col not in ['question_text', 'correct_answer', 'exam']]
        question_dict = {
            "question": row['question_text'],
            "options": [row[col] for col in options_cols if pd.notnull(row[col])],
            "answer": row['correct_answer'] if len(row['correct_answer']) == 1 else list(row['correct_answer']),
            "type": "single" if len(row['correct_answer']) == 1 else "multiple"
        }
        questions_list.append(question_dict)
    
    # Lưu vào file JSON
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(questions_list, f, ensure_ascii=False, indent=2)
    break